In [2]:
import numpy as np
import cv2
from keras.models import load_model, model_from_json

Using TensorFlow backend.
/home/utkarsh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/utkarsh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/utkarsh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/utkarsh/anaconda3/lib/python3.7/site-packages/t

In [9]:
def remove_color(image):
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    ret,grayscale=cv2.threshold(hsv[:,:,2],200,255,cv2.THRESH_BINARY)
    return grayscale


def horizontal(letter,length):
    size=(130-length)//2
    front=np.ones((150,size))*255
    back=np.ones((150,size+length%2))*255
    letter=np.hstack((front,letter,back))

    return letter

def segment(image):
    arr=np.unique(np.where(image==0)[1])
    start,end=arr[0],-1
    prev_end=0
    l=len(arr)
    chars=[]
    for i in range(l):
        if i==l-1:
            end=arr[i]
                
        elif arr[i+1]-arr[i]!=1:
            end=arr[i]
                
        if end!=-1:
            letter=image[:,start:end]
            letter=horizontal(letter,end-start)
            chars.append(letter)
                
            if i!=l-1:
                start=arr[i+1]
            end=-1
    chars = np.array(chars) 
    return len(chars),chars
def process_data(file):
    image = cv2.imread(file)
    gray_image = remove_color(image)
    return segment(gray_image)

In [16]:
def decaptcha( filenames ):
    numChars = 3 * np.ones(len(filenames))
    codes=[]
    with open('../model_num.json', 'r') as json_file:
        loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)
    model.load_weights("../model_num.h5")

    for i in range(len(filenames)):
        numChars[i], chars=process_data(filenames[i])
        code=''
        for image in chars:
            image = image/255
            image = image.reshape(1,150,130,1)
            prediction_one_hot = model.predict(image)
            prediction = chr(ord('A') + np.argmax(prediction_one_hot))
            code = code + prediction
        codes.append(code)
    codes = np.array(codes)
    return (numChars, codes)